In [ ]:
%matplotlib widget
# %matplotlib qt
from bmcs_shell.api import WBCell5Param, WBShellAnalysis, WBTessellation5PBeta, WBNumTessellation, WBNumTessellationInvest, WBTessellationBase, WBNumTessellationBase, WBCell5ParamBeta, WBTessellation4P, WBCell4Param
from bmcs_utils.api import Extruder
import numpy as np
import k3d
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def add_circle(plot, path, r, wireframe=False):
    n = 100
#     path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
    first_contour = Extruder.get_circle_points(r = r, n=n)[int(n/2):,:]

    extruder = Extruder(first_contour, path)
    vertices, indices = extruder.get_triangulation_vertices_and_indices(with_ends=False)
    
    # extruder.show_in_k3d_as_surface(with_ends=False)
    
    mesh = k3d.mesh(vertices, 
                    indices,
                    color=0xde2121,
                    opacity=0.2,
                    side='double')
    if wireframe:
        wf = k3d.lines(vertices, 
                          indices,
                          width = 35,
                          shader='mesh',
                          color=0xde2121)
        plot += wf
    plot += mesh

def add_ref_plane(plot):
    z = -6000
    size = 30000
    ref_plane = k3d.mesh([[size, size, z], [-size, size, z], [-size, -size, z],[size, -size, z]],
                         [[0, 1, 2], [2, 3, 0]],
                        side='double',
                        color = 0xe6e6e6)
    plot += ref_plane
    
def export_obj_file(wb_shell, name='wb_3d_print.obj'):
    I_Fi = wb_shell.I_Fi
    X_Ia = wb_shell.X_Ia / 1000
    
    # Write to obj file
    f = open(name, 'w')
    f.write('# Vertices: (' + str(X_Ia.shape[0]) + ')\n')
    for v in X_Ia:
        f.write('v ' + str(v)[1:-1] + '\n')
    f.write('\n# Tri Facets: (' + str(I_Fi.shape[0]) + ')\n')
    for ind in I_Fi + 1:
        f.write('f ' + str(ind)[1:-1] + '\n')
    f.close()

def get_span(wb_shell, n_mid_cells = 2):
    if n_mid_cells == 2:
        span_v = wb_shell.X_Ia[8] - wb_shell.X_Ia[1]
    elif n_mid_cells == 3:
        span_v = wb_shell.X_Ia[13] - wb_shell.X_Ia[1]
    elif n_mid_cells == 4:
        span_v = wb_shell.X_Ia[18] - wb_shell.X_Ia[1]
    elif n_mid_cells == 5:
        span_v = wb_shell.X_Ia[23] - wb_shell.X_Ia[1]
    return np.sqrt(span_v @ span_v)

def get_shell_height(wb_shell, n_mid_cells = 2):
    if n_mid_cells == 2:
        return wb_shell.X_Ia[3][2] - wb_shell.X_Ia[8][2]
    elif n_mid_cells == 3:
        return wb_shell.X_Ia[10][2] - wb_shell.X_Ia[13][2]
    elif n_mid_cells == 4:
        return wb_shell.X_Ia[8][2] - wb_shell.X_Ia[18][2]
    elif n_mid_cells == 5:
        return wb_shell.X_Ia[15][2] - wb_shell.X_Ia[23][2]
    
def get_shell_width(wb_shell, n_mid_cells = 2):
    # width of two cells (one cell in mid and two halves to sides)
    span_v = wb_shell.X_Ia[32 + (n_mid_cells-2)*8] - wb_shell.X_Ia[20 + (n_mid_cells-2)*5]
    return np.sqrt(span_v @ span_v)
    
def interp(interp_value, values, etas, zetas):
    try:
        f_eta = interp1d(values, etas, kind='linear')
        f_zeta = interp1d(values, zetas, kind='linear') # maybe try 'cubic' but it doesn't work for few values
        eta_inter = f_eta(interp_value)
        zeta_inter = f_zeta(interp_value)
    except:
        eta_inter, zeta_inter = np.nan, np.nan
    finally:
        return eta_inter, zeta_inter
    
def interp1(interp_value, values, y):
    try:
        f_y = interp1d(values, y, kind='linear')
        y_inter = f_y(interp_value)
    except:
        y_inter = np.nan
    finally:
        return y_inter
    
def round_to(value, base=5):
    return base * round(value/base)

def get_curv_angle(wb_cell):   
    X_Ia = wb_cell.X_Ia
    v_56 = (X_Ia[5] + X_Ia[6])/2
    v_12 = (X_Ia[1] + X_Ia[2])/2
    v_diff = v_12 - v_56
    oy_n = np.array([0, 1, 0])
    v_diff_n = v_diff / np.linalg.norm(v_diff)
    dot_product = np.dot(oy_n, v_diff_n)
    angle = np.arccos(dot_product)
    return np.rad2deg(angle)

#     r = 1000/angle # ra=s, radius*angle = curve
#     k = 1/r
#     return k
# get_curv_angle(wbt4p.wb_cell)

## Tested WB shell

In [ ]:
WBNumTessellation().interact()

In [ ]:
tested_wb_shell = dict(a =125,  b = 550, c = 175, gamma=np.deg2rad(46), n_phi_plus=3, n_x_plus=2, wireframe_width=5)
wbt4p = WBTessellation4P(**tested_wb_shell)
wbt4p.interact()

In [ ]:
get_shell_height(wbt4p, n_mid_cells=2)

In [ ]:
get_shell_width(wbt4p, n_mid_cells=3)

In [ ]:
get_span(wbt4p, n_mid_cells=2)

In [ ]:
wbt4p.wb_cell.R_0

In [ ]:
# export_obj_file(wbt4p, 'wb_tested2.obj')

## Parametric study

**We have: cross section height h, curvature k, span**

Cross section height h, is affected by c and gamma \
Curvature R, is affected by c and gamma

In [ ]:
# All eta, zeta combinations curves here give a span of 2118.16
from matplotlib import cm

def get_var_value(var, wbt4p, n_mid_cells):
    if var['name'] == 'span':
        return get_span(wbt4p, n_mid_cells=n_mid_cells) 
    elif var['name'] == 'height':
        return get_shell_height(wbt4p, n_mid_cells=n_mid_cells)
    elif var['name'] == 'width':
        return get_shell_width(wbt4p, n_mid_cells=n_mid_cells)
    elif var['name'] == 'R_0':
        return -wb_cell.R_0
    elif var['name'] == 'curv_angle':
        # for cell!!
        return get_curv_angle(wb_cell)

def get_data(n=50, 
             n_mid_cells=2, 
             a_range = [125],
             gamma_range=np.linspace(10, 85, 10), 
             var={'name':'span', 'value':2118.16},
             var2={'name':'height', 'value':279.54},
             var3={'name':'width', 'value':501.77},
            ):
    etas = np.concatenate((np.linspace(0, 1, int(n/2))[:-1], np.linspace(1, 10, int(n/2 + 1))))
    zetas = np.copy(etas)

    etas_grid, zetas_grid = np.meshgrid(etas, zetas)
    var_grid = np.zeros_like(etas_grid)

    z_angle = 7.39
    z_R_0 = 1000
    z_span = 2118.16

    fig, ax = plt.subplots()
    ax.set_title(var['name'] + '=' + str(var['value']) + ' contours')
    ax.set_xlabel(r'eta', fontsize=10)
    ax.set_ylabel(r'zeta', fontsize=10)

    fig_h, ax_h = plt.subplots()
    ax_h.set_title(var['name'] + '=' + str(var['value']))
    ax_h.set_ylabel(var2['name'])
    ax_h.set_xlabel('eta/zeta')
    ax_h.set_ylim(-1000, 5000)

    valid_params = []
    wbt4p = WBTessellation4P(n_phi_plus=n_mid_cells + 1, n_x_plus=2, wireframe_width=5)

    for i_a, a in enumerate(a_range):
        wbt4p.trait_set(a=a)
        
        valid_var1_2_params = []
        for i_gamma, gamma in enumerate(gamma_range):
            print(np.round((i_gamma + 1)/len(gamma_range) * 100, 1), '%, ', end='')
            
            wbt4p.trait_set(gamma=np.deg2rad(gamma))

            # Fill the grid of the variable
            # -------------------------------------------------------
            for i_eta in range(len(etas_grid)):
                for j_zeta in range(len(zetas_grid)):
                    eta = etas_grid[i_eta, j_zeta]
                    zeta = zetas_grid[i_eta, j_zeta]

                    wbt4p.trait_set(b = eta * a, c = zeta * a)

                    var_grid[i_eta, j_zeta] = get_var_value(var, wbt4p, n_mid_cells)

            # Plot 3d
            # --------
            if i_gamma == 0:
                fig_3d, ax_3d = plt.subplots(subplot_kw={"projection": "3d"})
                ax_3d.set_title('gamma = ' + str(round(gamma, 1)) + '°')
                ax_3d.set_xlabel(r'eta', fontsize=10)
                ax_3d.set_ylabel(r'zeta', fontsize=10)
                ax_3d.plot_surface(etas_grid, zetas_grid, var_grid,
                                       linewidth=0, antialiased=False, cmap=cm.coolwarm)

            # Find contour line corresponding to the variable value
            # -------------------------------------------------------
            color = np.random.rand(3, )
            # TODO: try scipy interp2d or interpn instead of getting data from contour 
            #  (however contour enables you to see if there are multiple solutions)
            cs = ax.contour(etas_grid, zetas_grid, var_grid, levels=[var['value']], colors=[color])

            for i, path in enumerate(cs.collections[0].get_paths()):
                length = len(path.vertices)
                if i==0:
                    longest_path = path
                elif length > len(longest_path.vertices):
                    longest_path = path

            path = longest_path # cs.collections[0].get_paths()[0] # longest_path
            eta_of_var = path.vertices[:, 0]
            zeta_of_var = path.vertices[:, 1]

            # Label every other level using strings
            ax.clabel(cs, inline=True, fmt={cs.levels[0]: '$\gamma$=' + str(round(gamma, 1))}, fontsize=10)

            print('path length=', len(path))

            # Find the possible shell heights consiering the fixed var value 
            # --------------------------------------------------------------
            var2_array = []
            for eta, zeta in zip(eta_of_var, zeta_of_var):
                wbt4p.trait_set(b = eta * a, c = zeta * a)
                var2_array.append(get_var_value(var2, wbt4p, n_mid_cells))

            ax_h.plot(eta_of_var, var2_array, '--', label='eta, $\gamma$=' + str(round(gamma, 1)), color=color)
            ax_h.plot(zeta_of_var, var2_array, label='zeta, $\gamma$=' + str(round(gamma, 1)), color=color)

            eta_inter, zeta_inter = interp(var2['value'], var2_array, eta_of_var, zeta_of_var)
            valid_var1_2_params.append([a, gamma, eta_inter, zeta_inter])

            ax_h.plot(eta_inter, var2['value'], 'o', color=color)
            ax_h.plot(zeta_inter, var2['value'], 'x', color=color)
    
        valid_var1_2_params = np.array(valid_var1_2_params)
        
        var3_array = []
        for params in valid_var1_2_params:
            a, gamma, eta, zeta = params
            wbt4p.trait_set(a=a, b=eta*a, c=zeta*a, gamma=np.deg2rad(gamma))
            var3_array.append(get_var_value(var3, wbt4p, n_mid_cells))

        gamma, eta, zeta = [interp1(var3['value'], var3_array, valid_var1_2_params[:, i]) for i in [1, 2, 3]]
        valid_params.append(dict(a=a,  b=a * eta, c=a * zeta, gamma=np.deg2rad(gamma), n_phi_plus=n_mid_cells + 1))
    
    ax_h.legend()
    fig_3d.show(), fig.show(), fig_h.show()
    
    print('valid_params=', valid_params)
    
    return valid_params

In [ ]:
# For generating final results:
valid_params = get_data(n=100, 
         n_mid_cells=4, 
         a_range = [100, 150],
         gamma_range=np.linspace(10, 85, 50), 
         var={'name':'span', 'value':2000},
         var2={'name':'height', 'value':300},
         var3={'name':'width', 'value':500}
        )

In [ ]:
# For generating final results:
valid_params = get_data(n=100, 
         n_mid_cells=4, 
         a_range = [100],
         gamma_range=np.linspace(10, 85, 50), 
         var={'name':'span', 'value':2000},
         var2={'name':'height', 'value':300},
         var3={'name':'width', 'value':500}
        )

### TODO: Pack ALL possible values in a up to 6 dimensions numpy array (design space array)

In [ ]:
path = np.array([[-500., 0., -1000.], [500., 0., -1000.]])
add_circle(wbt4p.pb.plot_fig, path = path, r=1000.)

In [ ]:
a = 100
wbt4p = WBTessellation4P(
#                          a=100,
#                          b = 5.492 * a, 
#                          c = 1.8466 * a, 
#                          gamma=np.deg2rad(54.53), 
#                          n_phi_plus=3, 
                        **valid_params_4[2],
                         n_x_plus=2, 
                         wireframe_width=5)
wbt4p.interact()

In [ ]:
# export_obj_file(wbt4p, 'span_2000_w_500_h_300_a_150_4_cells.obj')

In [ ]:
get_shell_height(wbt4p, n_mid_cells=4)

In [ ]:
get_shell_width(wbt4p, n_mid_cells=4)

In [ ]:
get_span(wbt4p, n_mid_cells=4)

In [ ]:
wbt4p.wb_cell.R_0

## Generating graphics

In [ ]:
# Cameras for k3d
# [
#  x1,y1,z1, # position of the camera in xyz space
#  x2,y2,z2, # the point where camera is currently looking at
#  x3,y3,z3  # orientation (up direction), this vector cannot be [0,0,0])
# ]
shell_perspective = [92989.64822524686, -70795.1623293042, 31459.457304330528,
                     0, 0, -202.457763671875,
                     -0.19436494432910192, 0.0869502251902479, 0.9770680256539523]
shell_front_view = [140000, -3000, -1000,
                    0,     0, -200,
                    0,     0,   1]
shell_top_view = [0, -10000, 140000, 0, 0, -200, -1, 0, 0]

### Tested WB shell

In [ ]:
wbt4p = WBTessellation4P(a =125,  b = 550, c = 175, gamma=np.deg2rad(46), n_phi_plus=3, n_x_plus=2, wireframe_width=4)
wbt4p.interact()
k3d_plot = wbt4p.pb.plot_fig

k3d_plot.camera_fov = 1
k3d_plot.screenshot_scale = 5.5
k3d_plot.grid_visible = False
k3d_plot.camera = shell_perspective
# k3d_plot.camera = shell_front_view
# k3d_plot.camera = shell_top_view

### Parametric study shells

In [ ]:
valid_params_2_4_cells = [
{'a': 100.0, 'b': 523.0972790827252, 'c': 185.53831578999467, 'gamma': 0.941503562154671, 'n_phi_plus': 3}, 
{'a': 150.0, 'b': 528.9869047532335, 'c': 188.42964733716423, 'gamma': 0.559480433250358, 'n_phi_plus': 3},
{'a': 100.0, 'b': 264.0443060812336, 'c': 152.74500755927858, 'gamma': 1.3808286675705175, 'n_phi_plus': 5},
{'a': 150.0, 'b': 267.29565489071757, 'c': 117.65995940135527, 'gamma': 1.015897789314392, 'n_phi_plus': 5}
]

In [ ]:
wbt4p = WBTessellation4P(
#                          a=100,
#                          b = 5.492 * a, 
#                          c = 1.8466 * a, 
#                          gamma=np.deg2rad(54.53), 
#                          n_phi_plus=3, 
                        **valid_params_2_4_cells[0],
                         n_x_plus=2, 
                         wireframe_width=4)
wbt4p.interact()
k3d_plot = wbt4p.pb.plot_fig

k3d_plot.camera_fov = 1
k3d_plot.screenshot_scale = 5.5
k3d_plot.grid_visible = False
# k3d_plot.camera = shell_perspective
# k3d_plot.camera = shell_front_view
k3d_plot.camera = shell_top_view

# add_circle(k3d_plot, path = np.array([[-4500, 0, -4120], [4500, 0, -4120]]), r=4070, wireframe=False)

## Attempt to get params for target geometry using Minimization

In [ ]:
tested_wb_shell = dict(a =a,  b = b, c = c, gamma=gamma, n_phi_plus=3, n_x_plus=2, wireframe_width=5)
wbt4p = WBTessellation4P(**tested_wb_shell)
wbt4p.interact()

In [ ]:
path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
add_circle(wbt4p.pb.plot_fig, path = path, r=4200)

In [ ]:
# Cell h in generic way
cell_X_Ia = wbt4p.wb_cell.X_Ia
mid_56 = (cell_X_Ia[5] + cell_X_Ia[6])/2
v_0_mid_56 = mid_56 - cell_X_Ia[0]
cell_h = np.sqrt(np.sum(v_0_mid_56 * v_0_mid_56))
cell_h

In [ ]:
# Cell h for symmetric cell with parameterization with O is origin
cell_h = wbt4p.wb_cell.X_Ia[5][2]
cell_h

In [ ]:
span = (wbt4p.X_Ia[1] - wbt4p.X_Ia[8])[1]
span

In [ ]:
print(wbt4p.wb_cell.symb.R_0)

In [ ]:
h = self.c * np.cos(gamma)

In [ ]:
get_structural_params_residual(125,  550, 175, np.deg2rad(46))

In [ ]:
from scipy.optimize import minimize

# def get_structural_params(wbt4p):
#     h = wbt4p.wb_cell.X_Ia[5][2]
#     k = 1/wbt4p.wb_cell.R_0
#     span = (wbt4p.X_Ia[1] - wbt4p.X_Ia[8])[1]
#     return h, k, span

def get_structural_params_residual(params):
    h_target = 121.5652 # mm
    k_target = 1/-2024.44 # 1/mm # R_0 = -1/2024.44
    span_target = 2118.1638 # mm
    R_0_target = -2024.44

    a, b, c, gamma = params
    h = c * np.cos(gamma)
    R_0 = a*(-tan(gamma) + 1/cos(gamma)) + sqrt(-a**2*(-tan(gamma) + 1/cos(gamma))**2 + b**2)*(-2*a*(-(a*tan(gamma) - a/cos(gamma) + c*cos(gamma))**2/(2*a*c*sin(gamma) - 2*a*c + b**2 + c**2*cos(gamma)**2) + 1)*tan(gamma) + 2*a*(-(a*tan(gamma) - a/cos(gamma) + c*cos(gamma))**2/(2*a*c*sin(gamma) - 2*a*c + b**2 + c**2*cos(gamma)**2) + 1)/cos(gamma) + a*(-tan(gamma) + 1/cos(gamma)) + a*tan(gamma) - a/cos(gamma) - c*cos(gamma) + sqrt(2*a**2*sin(gamma) + a**2*cos(gamma)**2 - 2*a**2 + b**2*cos(gamma)**2)*sin(2*np.arcsin((a*tan(gamma) - a/cos(gamma) + c*cos(gamma))/sqrt(2*a*c*sin(gamma) - 2*a*c + b**2 + c**2*cos(gamma)**2)))/np.abs(cos(gamma)))/(a*sin(2*np.arcsin((a*tan(gamma) - a/cos(gamma) + c*cos(gamma))/sqrt(2*a*c*sin(gamma) - 2*a*c + b**2 + c**2*cos(gamma)**2)))*tan(gamma) - a*sin(2*np.arcsin((a*tan(gamma) - a/cos(gamma) + c*cos(gamma))/sqrt(2*a*c*sin(gamma) - 2*a*c + b**2 + c**2*cos(gamma)**2)))/cos(gamma) - sqrt(-a**2*(-tan(gamma) + 1/cos(gamma))**2 + b**2) + sqrt(-a**2*sin(gamma)**2 + 2*a**2*sin(gamma) - a**2 - b**2*sin(gamma)**2 + b**2)*cos(2*np.arcsin((a*tan(gamma) - a/cos(gamma) + c*cos(gamma))/sqrt(2*a*c*sin(gamma) - 2*a*c + b**2 + c**2*cos(gamma)**2)))*np.abs(cos(gamma))/cos(gamma)**2)
    print('h, R0: ', h , R_0)
    print('Diff: ', h - h_target, R_0 - R_0_target)
    print('Result = ', np.abs(h - h_target) + np.abs(R_0 - R_0_target))
    return np.abs(h - h_target) + np.abs(R_0 - R_0_target)
#     return np.abs(R_0 - R_0_target)

def minimize_tesssel_params():
#     x0 = np.array([125.0,  550.0, 175.0, np.deg2rad(46)])
    x0 = np.array([500.0,  500.0, 100.0, np.deg2rad(20)])
#     x0 = np.arange(4)
#     try:
    res = minimize(get_structural_params_residual, x0, method='Nelder-Mead', tol=1e-2, bounds=((0.01, 1e10), (0.01, 1e10), (0.01, 1e10), (0.01, np.pi/2 - 0.01)))
#     except:
#         print('Error while minimizing!')
#         return np.array([0, 0])
    sol = res.x
    return sol

a, b, c, gamma = minimize_tesssel_params()
a, b, c, gamma

In [ ]:
wbt4p.wb_cell.R_0

## Visualizing

In [ ]:
# Cameras for k3d
# [
#  x1,y1,z1, # position of the camera in xyz space
#  x2,y2,z2, # the point where camera is currently looking at
#  x3,y3,z3  # orientation (up direction), this vector cannot be [0,0,0])
# ]
shell_side_view = [11206.956414346325, -9420.91749815528, 1858.3024602542291,
                   0.000244140625, -0.00048828125, -2129.4488372802734,
                  -0.20092161158887856, 0.08487577463430307, 0.9759234646614198]
cell_front_view = [0, -2000, 2000,
                   0,     0,   0,
                   0,     0,   1]
shell_front_view = [19000, 0, -1000,
                    0,     0, -2500,
                    0,     0,   1]
shell_top_view = [0, 0, 900000, 0, 0, -2500, -1, 0, 0]

In [ ]:
# FOV = 1
# Cameras for k3d
# [
#  x1,y1,z1, # position of the camera in xyz space
#  x2,y2,z2, # the point where camera is currently looking at
#  x3,y3,z3  # orientation (up direction), this vector cannot be [0,0,0])
# ]
shell_side_view = [591365.2482393435, -497120.07834650716, 208294.99640853348,
 0.000244140625, -0.00048828125, -2129.4488372802734, 
 -0.20092161158887856, 0.08487577463430307, 0.9759234646614198]

cell_front_view = [0, -2000, 2000,
                   0,     0,   0,
                   0,     0,   1]
shell_front_view = [800000, 0, -2000, 0, 0, -2000, 0, 0, 1]
shell_top_view = [0, 0, 900000, 0, 0, -2500, -1, 0, 0]

In [ ]:
path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
add_circle(wbt4p.pb.plot_fig, path = path, r=4200)

In [ ]:
# wbt4p.pb.plot_fig.camera = shell_front_view
wbt4p.pb.plot_fig.camera = shell_top_view
# wbt4p.pb.plot_fig.camera = shell_side_view
# wbt4p.pb.plot_fig.camera = cell_front_view

In [ ]:
wbt4p = WBTessellation4P(a = 500, b = 1000, c = 500, gamma = 1.0, n_phi_plus=7, n_x_plus = 5, wireframe_width=10)
wbt4p.interact()
wbt4p.pb.plot_fig.screenshot_scale = 5.5
wbt4p.pb.plot_fig.grid_visible = False
wbt4p.pb.plot_fig.camera = shell_side_view